### Day 19 - Not Enough Minerals
##### Part 1 - Sum of Quality Levels

Find the maximum number of geodes each blueprint can break open in 24 minutes. Sum up the quality level (#geodes x blueprint index) of each blueprint.

In [192]:
from collections import defaultdict

cache = defaultdict()
def dfs(timeRemaining, bots, materials, blueprint):
    global global_best
    global best_stats
    k = hash((timeRemaining, bots, materials, blueprint[0])) 
    if k in cache:                                
        return cache[k]
    
    if timeRemaining <= 1:           
        return materials[3] + bots[3] * timeRemaining

    # theoretical max
    tri = ((timeRemaining-1) * (timeRemaining)) // 2
    prune_check = materials[3] + bots[3] * timeRemaining + tri
    if prune_check < global_best[blueprint[0] - 1]:
        return -1 

    # can we make a geodebot?
    make_orebot, make_cbot, make_obsbot, make_gbot = True, True, True, True
    for i, mat in enumerate(materials[:-1]):
        if mat < blueprint[1][i]:
            make_orebot = False
        if mat < blueprint[2][i]:
            make_cbot = False
        if mat < blueprint[3][i]:
            make_obsbot = False
        if mat < blueprint[4][i]:
            make_gbot = False
            
    best = 0
    explored_flag = False       
        
    if make_gbot:
        if bots[0] >= blueprint[4][0] and bots[2] >= blueprint[4][2]:
            # triangle numbers - we can make a geode every turn 
            tri = ((timeRemaining) * (timeRemaining + 1)) // 2
            return materials[3] + bots[3] * timeRemaining + tri

        new_bots = tuple((bots[0], bots[1], bots[2], bots[3] + 1))
        new_materials = tuple((materials[0] + bots[0] - blueprint[4][0], materials[1] + bots[1] - blueprint[4][1], materials[2] + bots[2] - blueprint[4][2], materials[3] + bots[3]))
        best = dfs(timeRemaining - 1, new_bots, new_materials, blueprint)
        explored_flag = True
    else:
        if make_obsbot and bots[2] < blueprint[4][2]:
            #print(f"Make obsidianbot")
            new_bots = tuple((bots[0], bots[1], bots[2] + 1, bots[3]))
            build_materials = tuple((materials[0] + bots[0] - blueprint[3][0], materials[1] + bots[1] - blueprint[3][1], materials[2] + bots[2], materials[3] + bots[3]))
            best = max(best, dfs(timeRemaining - 1, new_bots, build_materials, blueprint))

            explored_flag = True
        if make_cbot and bots[1] < blueprint[3][1]:
            #print("Make claybot")
            new_bots = tuple((bots[0], bots[1]+1, bots[2], bots[3]))
            build_materials = tuple((materials[0] + bots[0] - blueprint[2][0], materials[1] + bots[1], materials[2] + bots[2], materials[3] + bots[3]))
            best = max(best, dfs(timeRemaining - 1, new_bots, build_materials, blueprint))
            explored_flag = True
        if make_orebot and (bots[0] < blueprint[4][0] or bots[0] < blueprint[3][0] or bots[0] < blueprint[2][0]):
            #print("Make orebot")
            new_bots = tuple((bots[0]+1, bots[1], bots[2], bots[3]))
            build_materials = tuple((materials[0] + bots[0] - blueprint[1][0], materials[1] + bots[1], materials[2] + bots[2], materials[3] + bots[3]))
            best = max(best, dfs(timeRemaining - 1, new_bots, build_materials, blueprint))
            explored_flag = True

    new_materials = tuple((materials[0] + bots[0], materials[1] + bots[1], materials[2] + bots[2], materials[3] + bots[3]))
    best = max(best, dfs(timeRemaining - 1, bots, new_materials, blueprint))

    if best >  global_best[blueprint[0] - 1]:
        global_best[blueprint[0] - 1] = best
        best_stats = tuple((timeRemaining, bots, materials, blueprint))
    cache[ hash((timeRemaining, bots, materials, blueprint[0])) ] = best
    return best


In [193]:
def get_blueprints(input_file):
    with open(input_file) as f:
        s = f.read().strip()
    blueprints = []
    for x in s.strip().split("\n") :
        l, r = x.split(":")
        idx = int(l.split(" ")[1])

        # costs are (ore, clay, obsidian)
        oreR = tuple((int(r.split(" costs ")[1].split(" ")[0]), 0, 0))
        clayR = tuple((int(r.split(" costs ")[2].split(" ")[0]), 0, 0))

        ll, rr = r.split(" costs ")[3].split(" ore and ")
        obsR = tuple((int(ll), int(rr.split(" ")[0]) , 0))

        lll, rrr = r.split(" costs ")[4].split(" ore and ")
        geodeR = tuple((int(lll), 0,int(rrr.split(" ")[0])))

        blueprints.append([idx, oreR, clayR, obsR, geodeR])

    return blueprints



In [194]:
# This gets the correct answer, but takes 52s to finish. Going to try to optimize this a bit after reading part 2. 

res = 0
blueprints = get_blueprints("aoc_19_input.txt")
global_best = [0 for _ in range(len(blueprints))]
for i, bp in enumerate(blueprints):
    bp_dfs = dfs(24, (1,0,0,0), (0,0,0,0), bp)
    quality_level = bp[0] * bp_dfs
    res += quality_level
print(f"Part 1: Sum of all quality levels: {res}")


Part 1: Sum of all quality levels: 960


##### Part 2 - Eaten Blueprints

Find the product of the number of geodes found with the first three blueprints.


In [195]:
from functools import reduce 

# This takes about 43s. Not ideal, but works for now. 

res_part2 = []
global_best = [0 for _ in range(len(blueprints))]
for bp in blueprints[:3]:
    prod_term = dfs(32, (1,0,0,0), (0,0,0,0), bp)
    res_part2.append(prod_term)

print(f"Part 2: The first three blueprints yield {res_part2} after 32 seconds. Their product is: {reduce(lambda a,b: a*b, res_part2)}")

Part 2: The first three blueprints yield [30, 4, 17] after 32 seconds. Their product is: 2040
